In [26]:
# | default_exp classes.DomoPublish

In [49]:
# | exporti
from dataclasses import dataclass, field
from typing import Optional, List

import datetime as dt
import pandas as pd
import asyncio
import httpx

import uuid

from nbdev.showdoc import patch_to

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.publish as publish_routes

import domolibrary.classes.DomoLineage as dmdl

import domolibrary.utils.chunk_execution as ce

In [28]:
#| hide

import os


# Publish

In [29]:
# | export
@dataclass
class DomoPublication_Subscription:
    subscription_id: str
    publication_id: str
    domain: str
    created_dt: Optional[dt.datetime] = None

    @classmethod
    def _from_json(cls, json):

        dd = json
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(json)

        return cls(
            subscription_id=dd.id or dd.subscriptionId,
            publication_id=dd.publicationId,
            domain=dd.domain or dd.publisherDomain,
            created_dt=(
                dt.datetime.fromtimestamp(dd.created / 1000) if dd.created else None
            ),
        )

In [30]:
# | hide

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await publish_routes.get_publication_by_id(
    publication_id=publication_id, auth=token_auth
)

# res.response
for child in res.response["children"]:
    print(child["content"])
# subscriber_ls = res.response.get("subscriptionAuthorizations")

# [
#     DomoPublication_Subscription._from_json(subscriber).__dict__
#     for subscriber in subscriber_ls
# ]

warning this token has not been validated by who_am_i, run get_auth_token first
adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 16, 'module_index': 12, 'num_stacks_to_drop_passed': 3}
{'id': '891a25be-30b3-4b77-bbc4-18c4619417c4', 'domain': 'domo-community.domo.com', 'customerId': '', 'userId': '1893952720', 'domoObjectId': '1880852133', 'type': 'PAGE', 'updated': 1711219377788, 'hash': 'e783def6-b6d2-4188-bb67-f161db8e107b'}
{'id': '3ec590ae-641c-4a2d-b0af-632448479f03', 'domain': 'domo-community.domo.com', 'customerId': '', 'userId': '1893952720', 'domoObjectId': 'ea859b06-5dbe-4c31-be76-de921447d550', 'type': 'DATASET', 'updated': 1711219377788, 'hash': '065d8f18-0882-4266-a21c-19ade9ce32ab'}
{'id': '011bd6fc-4713-42cd-9ae3-fc92ab1352e3', 'domain': 'domo-community.domo.com', 'customerId': '', 'userId': '1893952720', 'domoObjectId': 'cf149711-df55-4dc3-a159-59a26d066baa', 'type': 'DATASET', 'updated': 1711219377788, 'hash': '568de3d8-8c95-42b2-8d

In [31]:
# | export
@dataclass
class DomoPublication_Content:
    content_id: str
    entity_type: str
    entity_id: str
    entity_domain: str
    is_v2: bool
    is_direct_content: bool

    @classmethod
    def _from_json(cls, obj: dict):

        dd = obj
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(obj)

        dmpc = cls(
            content_id=dd.id,
            entity_type=dd.content.type,
            entity_id=dd.content.domoObjectId,
            entity_domain=dd.content.domain,
            is_v2=dd.isV2,
            is_direct_content=dd.useDirectContent,
        )

        return dmpc

    def to_api_json(self):
        temp_dict = {
            "domain": self.entity_domain,
            "domoObjectId": self.entity_id,
            "customerId": self.entity_domain,
            "type": self.entity_type,
        }
        return temp_dict

In [32]:
# | hide

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await publish_routes.get_publication_by_id(
    publication_id=publication_id, auth=token_auth
)

if res.is_success:
    content_ls = res.response.get("children")
    # print(content_ls[0])

    pd.DataFrame(
        [DomoPublication_Content._from_json(content) for content in content_ls]
    )

warning this token has not been validated by who_am_i, run get_auth_token first
adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 16, 'module_index': 12, 'num_stacks_to_drop_passed': 3}


In [33]:
# | export
class DomoPublication_UnexpectedContentType(Exception):
    def __init__(self, publication_id, content_type, domo_instance):
        super().__init__(
            f"DomoPublication_Instantiation: Unexpected content type {content_type} in publication {publication_id} in {domo_instance}"
        )

In [34]:
# | export
@dataclass
class DomoPublication:
    id: str
    name: str
    description: str
    is_v2: bool
    created_dt: dt.datetime

    auth: dmda.DomoAuth = field(default=None, repr=False)

    subscription_authorizations: List[DomoPublication_Subscription] = field(
        default_factory=list
    )
    content: List[DomoPublication_Content] = field(default_factory=list)

    content_page_id_ls: List[str] = field(default_factory=list)
    content_dataset_id_ls: List[str] = field(default_factory=list)

    lineage: dmdl.DomoLineage = None

    def __post_init__(self):
        self.lineage = dmdl.DomoLineage(parent=self)

    @classmethod
    def _from_json(cls, obj, auth: dmda.DomoAuth = None):

        dd = util_dd.DictDot(obj)

        domo_pub = cls(
            id=dd.id,
            name=dd.name,
            description=dd.description,
            created_dt=(
                dt.datetime.fromtimestamp(dd.created / 1000) if dd.created else None
            ),
            is_v2=dd.isV2,
            auth=auth,
        )

        if dd.subscriptionAuthorizations and len(dd.subscriptionAuthorizations) > 0:
            domo_pub.subscription_authorizations = [
                DomoPublication_Subscription._from_json(sub)
                for sub in dd.subscriptionAuthorizations
            ]

        # publish only supports sharing pages and datasets
        if dd.children and len(dd.children) > 0:
            for child in dd.children:

                dmpc = DomoPublication_Content._from_json(child)
                domo_pub.content.append(dmpc)

                if dmpc.entity_type == "PAGE":
                    domo_pub.content_page_id_ls.append(dmpc.entity_id)

                elif dmpc.entity_type == "DATASET":
                    domo_pub.content_dataset_id_ls.append(dmpc.entity_id)

                else:
                    raise DomoPublication_UnexpectedContentType(
                        publication_id=domo_pub.id,
                        content_type=dmpc.entity_type,
                        domo_instance=auth.domo_instance,
                    )

        return domo_pub

In [35]:
# | export
@patch_to(DomoPublication, cls_method=True)
async def get_from_id(cls, publication_id=None, auth: dmda.DomoAuth = None, timeout=10):

    auth = auth or cls.auth

    publication_id = publication_id or cls.publication_id

    res = await publish_routes.get_publication_by_id(
        auth=auth, publication_id=publication_id, timeout=timeout
    )

    if not res.is_success:
        return None

    return cls._from_json(obj=res.response, auth=auth)

#### sample implementation of get_from_id


In [36]:
#| eval : false

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_publication = await DomoPublication.get_from_id(
    publication_id=publication_id, auth=token_auth
)

from pprint import pprint

pprint(domo_publication)

warning this token has not been validated by who_am_i, run get_auth_token first
DomoPublication(id='701eb547-4244-43f6-bcda-003ed2c483c7',
                name='test_publish',
                description='',
                is_v2=True,
                created_dt=datetime.datetime(2023, 4, 27, 1, 36, 20, 642000),
                subscription_authorizations=[DomoPublication_Subscription(subscription_id='447bf630-7113-4323-bf67-773dcc450a3a',
                                                                          publication_id='701eb547-4244-43f6-bcda-003ed2c483c7',
                                                                          domain='test.domo.com',
                                                                          created_dt=datetime.datetime(2023, 4, 27, 1, 36, 20, 642000))],
                content=[DomoPublication_Content(content_id='cb257f38-6d6b-46b7-92c5-aee90fde7b66',
                                                 entity_type='PAGE',
                      

# Subscription

# DomoPublications

In [37]:
# | export


@dataclass
class DomoPublications:
    @classmethod
    async def get_subscription_summaries(
        cls,
        auth: dmda.DomoAuth,
        session: httpx.AsyncClient = None,
        return_raw: bool = False,
        debug_api: bool = False,
    ):
        """get instances subscription summaries"""

        res = await publish_routes.get_subscription_summaries(
            auth=auth, session=session, debug_api=debug_api
        )

        if return_raw:
            return res

        if not res.is_success:
            return res

        sub_ls = res.response

        return [DomoPublication_Subscription._from_json(sub) for sub in sub_ls]

#### sample implementation of get_subscription_summaries

In [38]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoPublications.get_subscription_summaries(auth=token_auth, return_raw=False)

warning this token has not been validated by who_am_i, run get_auth_token first


[]

In [39]:
# | export
@patch_to(DomoPublications, cls_method=True)
async def search_publications(
    cls: DomoPublications,
    auth=dmda.DomoAuth,
    search_term: str = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
):
    res = await publish_routes.search_publications(
        auth=auth,
        search_term=search_term,
        session=session,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    if not res.is_success or (res.is_success and len(res.response) == 0):
        return None

    return await ce.gather_with_concurrency(
        n=60,
        *[
            DomoPublication.get_from_id(publication_id=sub_obj["id"], auth=auth)
            for sub_obj in res.response
        ]
    )

In [40]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

(await DomoPublications.search_publications(auth=token_auth, return_raw=False))[0:5]

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoPublication(id='095426a1-2c35-468b-bb96-36c516382f19', name='Ellibot', description='', is_v2=True, created_dt=datetime.datetime(2023, 11, 10, 11, 5, 10, 961000), subscription_authorizations=[DomoPublication_Subscription(subscription_id='173ee64d-4f68-4d40-b634-22b5332f4e45', publication_id='095426a1-2c35-468b-bb96-36c516382f19', domain='domo-community-ellibot.domo.com', created_dt=datetime.datetime(2023, 11, 10, 11, 5, 10, 961000))], content=[DomoPublication_Content(content_id='c61531b6-81ac-45ec-9a77-d41194406097', entity_type='DATASET', entity_id='f6210193-5322-4481-a1de-d07aed544050', entity_domain='domo-community.domo.com', is_v2=True, is_direct_content=True)], content_page_id_ls=[], content_dataset_id_ls=['f6210193-5322-4481-a1de-d07aed544050'], lineage=DomoLineage(page_id_ls=[], card_id_ls=[], dataflow_id_ls=[], dataset_id_ls=[], entity_ls=[])),
 DomoPublication(id='99a4f772-f0e0-47a6-873a-5870d03b9134', name='Landing Page QA', description='', is_v2=True, created_dt=datetime

In [41]:
# | export


@patch_to(DomoPublication, cls_method=False)
def convert_content_to_dataframe(self, return_raw: bool = False):

    output_ls = [
        {
            "plubication_id": self.id,
            "publication_name": self.name,
            "is_v2": self.is_v2,
            "publish_created_dt": self.created_dt,
            "entity_type": row.type,
            "entity_id": row.id,
        }
        for row in self.content_entity_ls
    ]

    if return_raw:
        return output_ls

    return pd.DataFrame(output_ls)


@patch_to(DomoPublication, cls_method=False)
def convert_lineage_to_dataframe(self, return_raw: bool = False):

    flat_lineage_ls = self.lineage._flatten_lineage()

    output_ls = [
        {
            "plubication_id": self.id,
            "publication_name": self.name,
            "is_v2": self.is_v2,
            "publish_created_dt": self.created_dt,
            "entity_type": row.get("entity_type"),
            "entity_id": row.get("entity_id"),
        }
        for row in flat_lineage_ls
    ]

    if return_raw:
        return output_ls

    return pd.DataFrame(output_ls)

In [42]:
# | export
@patch_to(DomoPublication, cls_method=True)
async def create_publication(
    cls,
    name: str,
    content_ls: List[DomoPublication_Content],
    subscription_ls: List[DomoPublication_Subscription],
    unique_id: str = None,
    description: str = None,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
):

    if not isinstance(subscription_ls, list):
        subscription_ls = [subscription_ls]

    auth = auth or cls.auth
    domain_ls = []
    content_json_ls = []
    for sub in subscription_ls:
        domain_ls.append(sub.domain)
    for content_item in content_ls:
        content_json_ls.append(content_item.to_api_json())

    if not unique_id:
        unique_id = str(uuid.uuid4())
    if not description:
        description = ""

    body = publish_routes.generate_publish_body(
        url=f"{auth.domo_instance}.domo.com",
        sub_domain_ls=domain_ls,
        content_ls=content_json_ls,
        name=name,
        unique_id=unique_id,
        description=description,
        is_new=True,
    )

    res = await publish_routes.create_publish_job(auth=auth, body=body)
    if debug_api:
        print("Create the new Publish job")
    if res.status != 200:
        print(res)
        await asyncio.sleep(2)
        res = await publish_routes.get_publication_by_id(
            auth=auth, publication_id=unique_id
        )
        if res.status != 200:
            return None
        else:
            return cls._from_json(obj=res.response, auth=auth)

    return cls._from_json(obj=res.response, auth=auth)

In [43]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

content_ls = []
content_item = DomoPublication_Content(
    content_id="",
    entity_type="DATASET",
    entity_id="e17fd398-8abf-4cb5-8ed1-5394e274fbf4",
    entity_domain="domo-community.domo.com",
    is_v2=True,
    is_direct_content=True,
)
content_ls.append(content_item)

sub_ls = []
sub_item = DomoPublication_Subscription(
    subscription_id="",
    publication_id="",
    domain="test.domo.com",
    created_dt=dt.datetime.now(),
)
sub_ls.append(sub_item)

# await DomoPublication.create_publication(auth = token_auth, name="Test OZ",
#                                  content_ls=content_ls,
#                                  subscription_ls=sub_ls)

In [44]:
# | export
@patch_to(DomoPublication, cls_method=True)
async def update_publication(
    cls,
    name: str,
    content_ls: List[DomoPublication_Content],
    subscription_ls: List[DomoPublication_Subscription],
    publication_id: str,
    description: str = None,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
):

    if not isinstance(subscription_ls, list):
        subscription_ls = [subscription_ls]

    auth = auth or cls.auth
    domain_ls = []
    content_json_ls = []
    for sub in subscription_ls:
        domain_ls.append(sub.domain)
    for content_item in content_ls:
        content_json_ls.append(content_item.to_api_json())

    if not description:
        description = ""
    body = publish_routes.generate_publish_body(
        url=f"{auth.domo_instance}.domo.com",
        sub_domain_ls=domain_ls,
        content_ls=content_json_ls,
        name=name,
        unique_id=publication_id,
        description=description,
        is_new=False,
    )

    res = await publish_routes.udpate_publish_job(
        auth=auth, publication_id=publication_id, body=body
    )
    if debug_api:
        print("Update Publish job by id")
    if res.status != 200:
        print(res)
        await asyncio.sleep(2)
        res = await publish_routes.get_publication_by_id(
            auth=auth, publication_id=publication_id
        )
        if res.status != 200:
            return None
        else:
            return cls._from_json(obj=res.response, auth=auth)

    return cls._from_json(obj=res.response, auth=auth)

In [45]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
publication_id = "438731a1-7e4e-4863-967f-fcfad22c9247"
domo_publication = await DomoPublication.get_from_id(
    publication_id=publication_id, auth=token_auth
)

await DomoPublication.update_publication(
    auth=token_auth,
    name="Test OZ - updated",
    publication_id=publication_id,
    content_ls=domo_publication.content,
    subscription_ls=domo_publication.subscription_authorizations,
)

warning this token has not been validated by who_am_i, run get_auth_token first
ResponseGetData(status=202, response={'name': 'Test OZ - updated', 'description': '', 'domain': 'domo-community.domo.com', 'content': [{'id': '5b49b7a5-8bad-4531-845e-aea6a600248e', 'domain': 'domo-community.domo.com', 'customerId': 'domo-community.domo.com', 'userId': 'Fake User Id', 'domoObjectId': '384424178', 'type': 'PAGE', 'updated': 1711219382904, 'hash': '01d7c069-9912-4948-9b2f-4a1223d944f7'}, {'id': '23543278-a0e3-4f68-a9cc-15931d1d81f9', 'domain': 'domo-community.domo.com', 'customerId': 'domo-community.domo.com', 'userId': 'Fake User Id', 'domoObjectId': '4ef43af5-67e7-4b9e-bd58-c4e592aa289a', 'type': 'DATASET', 'updated': 1711219382904, 'hash': '73dfb293-73c9-4ea9-b8e3-755059f6c375'}, {'id': '92856ce0-4c33-41c1-bbd6-b89f94e42867', 'domain': 'domo-community.domo.com', 'customerId': 'domo-community.domo.com', 'userId': 'Fake User Id', 'domoObjectId': 'e17fd398-8abf-4cb5-8ed1-5394e274fbf4', 'type'

DomoPublication(id='438731a1-7e4e-4863-967f-fcfad22c9247', name='Test OZ - updated', description='', is_v2=True, created_dt=datetime.datetime(2023, 5, 2, 15, 6, 24, 358000), subscription_authorizations=[DomoPublication_Subscription(subscription_id='a0272907-d34c-4c27-b716-f2386dfecd19', publication_id='438731a1-7e4e-4863-967f-fcfad22c9247', domain='test.domo.com', created_dt=datetime.datetime(2023, 5, 2, 15, 6, 24, 358000))], content=[DomoPublication_Content(content_id='16e1b47f-6677-41a3-9c80-894550fec649', entity_type='PAGE', entity_id='384424178', entity_domain='domo-community.domo.com', is_v2=True, is_direct_content=True), DomoPublication_Content(content_id='14d1a1e9-8673-4b3f-aa72-1d32a6041140', entity_type='DATASET', entity_id='4ef43af5-67e7-4b9e-bd58-c4e592aa289a', entity_domain='domo-community.domo.com', is_v2=True, is_direct_content=True), DomoPublication_Content(content_id='53ab2b71-7b4c-47c9-9d0f-5deb3c7974f8', entity_type='DATASET', entity_id='e17fd398-8abf-4cb5-8ed1-5394e2

In [46]:
# | export
@patch_to(DomoPublication, cls_method=True)
async def get_subscription_invites_list(
    cls, auth: dmda.DomoAuth, debug_api: bool = False
):

    res = await publish_routes.get_subscription_invititations(
        auth=auth, debug_api=debug_api
    )
    if debug_api:
        print("Getting Publish subscription invites")

    if res.status == 200:
        return res.response
    else:
        return None


# | export
@patch_to(DomoPublication, cls_method=True)
async def accept_invite_by_id(
    cls, auth: dmda.DomoAuth, subscription_id: str, debug_api: bool = False
):

    res = await publish_routes.accept_invite_by_id(
        auth=auth, subscription_id=subscription_id, debug_api=debug_api
    )
    if debug_api:
        print(f"Accept invite by id {subscription_id}")

    if res.status == 200:
        return res.response
    else:
        return None

In [47]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoPublication.get_subscription_invites_list(auth=token_auth)

warning this token has not been validated by who_am_i, run get_auth_token first


[]

In [48]:
# | hide

import nbdev

nbdev.nbdev_export()